## 1. 请写一下TF-IDF的计算公式

TF（Term Frequency，词频）表示一个给定词语t在一篇给定文档d中出现的频率。TF越高，则词语t对文档d来说越重要，
TF越低，则词语t对文档d来说越不重要。

IDF（Inverse Document Frequency，逆向文件频率）的主要思想是：如果包含词语t的文档越少，则IDF越大，
说明词语t在整个文档集层面上具有很好的类别区分能力。

对于在某一文档 dj 里的词语 ti 来说，ti 的词频可表示为：

TF: tf(i,j) = n(i,j)/∑k n(k,j)

其中 ni,j 是词语 ti 在文档 dj 中的出现次数，分母则是在文件 dj 中所有词语的出现次数之和。

某一特定词语的IDF，可以由总文件数除以包含该词语的文件数，再将得到的商取对数得到：

IDF:  idf(i) = log(10) |D| / | {j: t(i) ∈ d(j)} |

其中 |D| 是语料库中所有文档总数，分母是包含词语 ti 的所有文档数。



## 2. LDA算法的基本假设是什么？



LDA假设文档主题的先验分布是Dirichlet分布,假设主题中词的先验分布是Dirichlet分布(数学推导繁琐，自行学习，不做展示）


## 3. 在TextRank算法中构建图的权重是如何得到的？


考察jieba源码后，发现其权重是将共现词典中的词i，词j作为一条边起始点和终止点，共现的次数作为边的权重。
共现词典的构建则通过滑动窗口，如（1，2，3） ——> (1,2),(1,3),(2,3),该例子滑动窗口为2。

当然，也可以结合word2vec词向量，通过比较词i和词j的相似度，作为边的权重。

## 4. 什么是命名实体识别？ 有什么应用场景？



命名实体识别（Named Entity Recognition，简称NER），又称作“专名识别”，是指识别文本中具有特定意义的实体，主要包括人名、地名、机构名、专有名词等。简单的讲，就是识别自然文本中的实体指称的边界和类别。

命名实体识别是信息提取、问答系统、句法分析、机器翻译、面向Semantic Web的元数据标注等应用领域的重要基础工具，在自然语言处理技术走向实用化的过程中占有重要地位。

## 5.NLP主要有哪几类任务 ？


1. 序列标注：分词/POS Tag(词性标注）/NER/语义标注
2. 分类任务：文本分类/情感计算
3. 句子关系判断：Entailment/QA/自然语言推理
4. 生成式任务：机器翻译/文本摘要


In [ ]:
import pandas as pd
import numpy as np
import jieba
import re
from collections import defaultdict
import sys
from operator import itemgetter
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec

## 构造text-rank抽取关键词

# 数据准备

In [13]:
han_filename = "/Users/junjiexie/Documents/NLP学习/nlp文本摘要项目/sqlResult_1558435.csv"
data = pd.read_csv(han_filename,encoding="GB18030")
articles = data["content"].tolist()

In [14]:
def get_stopwords():
    stopwords = []
    with open(r"/Users/junjiexie/Documents/NLP学习/nlp第九课/停用词表.txt" ,encoding="utf-8") as f:
        line_str = f.readline()
        while line_str != "":
            line_str = line_str.strip()
            stopwords.append(line_str)
            line_str = f.readline()
    return set(stopwords)

def token(string):return re.findall('\w+', string)

In [15]:
def sentences_deal(sentences):
    output_list = []
    input = "".join(token(sentences))
    cut_list = ",".join(jieba.cut(input)).split(",")
    
    stopwords = get_stopwords()
    for str in cut_list:
        if str in stopwords:
            continue
        else:
            output_list.append(str)
    
    return output_list
        

In [48]:
# 定义无向有权图
class UndirectWeightedGraph:
    d = 0.85
 
    def __init__(self):
        self.graph = defaultdict(list)
    
    #有权无向图的数据结构
    def addEdge(self, start, end, weight):
        # use a tuple (start, end, weight) instead of a Edge object
        self.graph[start].append((start, end, weight))
        self.graph[end].append((end, start, weight))
 
    def rank(self):
        #记录结点权值
        ws = defaultdict(float)
        #记录结点出度和
        outSum = defaultdict(float)
        
        # 初始化各个结点的权值
        wsdef = 1.0 / (len(self.graph) or 1.0)
        
        # 统计各个结点的出度的次数之和
        for n, out in self.graph.items():
            ws[n] = wsdef
            outSum[n] = sum((e[2] for e in out), 0.0)
 
        # this line for build stable iteration
        sorted_keys = sorted(self.graph.keys())
        # 遍历若干次，保证权值收敛，这里写了100次
        for x in range(100):  
            for n in sorted_keys:
                s = 0
                # 将这些入度结点贡献后的权值相加
                # 贡献率 = 入度结点与结点n的共现次数 / 入度结点的所有出度的次数
                for e in self.graph[n]:
                    s += e[2] / outSum[e[1]] * ws[e[1]]
                # 更新结点n的权值
                ws[n] = (1 - self.d) + self.d * s
 
        (min_rank, max_rank) = (sys.float_info[0], sys.float_info[3])
 
        for w in ws.values():
            if w < min_rank:
                min_rank = w
            if w > max_rank:
                max_rank = w
        
        #权值归一化，修正数值分布
        for n, w in ws.items():
            ws[n] = (w - min_rank / 10.0) / (max_rank - min_rank / 10.0)
 
        return ws

In [49]:
def cosine_similarity(x, y, norm=False):
    """ 计算两个向量x和y的余弦相似度 """
    assert len(x) == len(y), "len(x) != len(y)"
    zero_list = [0] * len(x)
    if list(x) == zero_list or list(y) == zero_list:
        return float(1) if list(x) == list(y) else float(0)

    # method 1
    res = np.array([[x[i] * y[i], x[i] * x[i], y[i] * y[i]] for i in range(len(x))])
    cos = sum(res[:, 0]) / (np.sqrt(sum(res[:, 1])) * np.sqrt(sum(res[:, 2])))

    return 0.5 * cos + 0.5 if norm else cos  # 归一化到[0, 1]区间内

def word_similarity(model, word1, word2):

    #有可能有些词是不在word2vec里的，因此无法计算,给出一个接近零的相似度
    try:
        word1_vec = model[word1]
    except KeyError:
        return 0.001
    try:  
        word2_vec = model[word2]
    except KeyError:
        return 0.001
    
    return cosine_similarity(word1_vec, word2_vec)

In [50]:
def textrank(sentences, topK=10, span_num=2):
    # 导入word2vec
    path = '/Users/junjiexie/OursRepository/text-abstract-extraction/Data/wiki_han_word2vec_300维度.model'
    model = Word2Vec.load(path)
    # 定义无向有权图
    g = UndirectWeightedGraph()
    # 定义权重词典
    cm = defaultdict(int)
    # 文本预处理
    words = sentences_deal(sentences)
    # 依次遍历每个词
    for i, wp in enumerate(words):
            # 依次遍历词i 之后窗口范围内的词
        for j in range(i + 1, i + span_num):
            # 词j 不能超出整个句子
            if j >= len(words):
                break
            #判断这个词组是否已经出现过
            if cm[(wp, words[j])] == 0:
                cm[(wp, words[j])] = word_similarity(model=model, word1=wp, word2=words[j])
            else:
                continue
    
    # jieba中对权重的定义是两词共现次数，这里换成word2vec词向量相似度
    for terms, w in cm.items():
        g.addEdge(terms[0], terms[1], w)
    
    # 运行text-rank算法
    nodes_rank = g.rank()
    
    # 根据指标值进行排序
    tags = sorted(nodes_rank.items(), key=itemgetter(1), reverse=True)
 
    # 输出topK个词作为关键词
    if topK:
        return tags[:topK]
    else:
        return tags



In [52]:
textrank(articles[0])

#感觉用word2vec相似度作为权重有点怪怪的

/Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[('发布', 1.0),
 ('更新', 0.6395818438752795),
 ('月', 0.2810899844929091),
 ('本周', 0.18849534996885373),
 ('含', 0.15672863164471557),
 ('开发', 0.12142483512437663),
 ('更新换代', 0.11155062740108165),
 ('版', 0.09465604762329465),
 ('去年', 0.09103145940670385),
 ('外', 0.08444310231583688)]

In [53]:
textrank(articles[1])


/Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[('进入', 1.0),
 ('澎湃', 0.5847296259939706),
 ('拿到', 0.0548973629047256),
 ('不会', 0.0546180254984679),
 ('强调', 0.05460621538170052),
 ('考虑', 0.05459062935145518),
 ('PCB', 0.0545797095417014),
 ('空间', 0.05457955167302171),
 ('30', 0.054579460817448286),
 ('按计划', 0.05457935287802686)]

In [54]:
textrank(articles[2])



/Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[('手机', 1.0),
 ('缩水', 0.29252452749916613),
 ('大屏', 0.2602298390761203),
 ('拥有', 0.21514176678840113),
 ('旗舰', 0.20898436679888105),
 ('AMOLED', 0.20599958350122996),
 ('虎', 0.2046873581381779),
 ('可能', 0.20028676493912392),
 ('应该', 0.19264664010165564),
 ('掌握', 0.19159077476197756)]

In [55]:
# 完成选做一

